In [ ]:
%load_ext autoreload
%matplotlib inline

In [ ]:
%autoreload 2

import sys,os,glob
import psutil

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from pprint import pprint
from skimage import measure
from scipy.optimize import curve_fit

from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs
from datetime import datetime as dt
import pickle

import datetime as dt
import re
import gc
import matplotlib.animation as animation
import warnings

## current script object
thismodule = sys.modules[__name__]

# Own settings
from settings import *

workdir = os.getcwd()
moduledir, fcndir = defineDir(workdir)

In [ ]:
%autoreload 2

# to access segmentation files and simulation outputs
from fcns_load_DYAMOND_SAM import *
# to access TOOCAN objects
from load_TOOCAN_DYAMOND_modif_BF import *
# mapping function
from lifecycle_mapping import *
# display functions
from fcns_movie import *
# plotting
from plot1D import *

In [ ]:
# load relation table
df = relation_table = loadRelTable('DYAMOND_SEG')

In [ ]:
glob.glob(os.path.join(DIR_TOOCAN_DYAMOND,'*.gz'))

In [ ]:
toocan = loadAllMCSs(DIR_TOOCAN_DYAMOND,load_TOOCAN_DYAMOND)

In [ ]:
# list of TOOCAN labels, for quicker mapping on toocan list
labels_toocan = [toocan[i].label for i in range(len(toocan))]

In [ ]:
# list of TOOCAN labels, for quicker mapping on toocan list
labels_toocan = [toocan[i].label for i in range(len(toocan))]

# LifeCycleMapping object

In [ ]:
# set location on disk and check if exists
timetable_file = 'TOOCAN_time_table.csv'
timetable_path = os.path.join(DIR_DATA,timetable_file)

# check if table exists on disk
table_exists = len(glob.glob(timetable_path)) > 0 

# choose to overwrite
overwrite = False

# Load or compute time table

if table_exists and not overwrite:
    
    #-- read from disk
    print('reading from %s'%timetable_path)
    toocan_timetable = pd.read_csv(timetable_path)
    
else:
    
     toocan_timetable = None


In [ ]:
# initialize LCM object
lcm = LifeCycleMapping(relation_table,toocan_timetable,toocan)

## test defineMappingLabelToIndex()

In [ ]:
# compute mapping label to index for faster fetching of MCS in toocan list by label
lcm.defineMappingLabelToIndex()

In [ ]:
##-- unit test for random labels

label = 200 # pick any valid label here
print('label:',label)

ind_label = lcm.toocan_index_of_label[label]
print('index:',ind_label)

print('same as input label?',labels_toocan[ind_label] == label)

In [ ]:
##-- Detail of the defineMappingLabelToIndex method, line by line

# ex_labels_toocan = [1,3,4,5,7,8,9,10]
ex_labels_toocan = np.array([10,3,9,8,7,5,1,4])
# max_lab_toocan = 
ex_labels_valid = [3,5,7,8,9]

# 0. initiate with nans
# inds_labels_all = np.array(np.arange(0,np.nanmax(self.labels_toocan)+1),dtype=float)
inds_labels_all = np.array(np.arange(0,np.nanmax(ex_labels_toocan)+1),dtype=float)
print('inds_labels_all:',inds_labels_all)
inds_labels_all[0] = np.nan
print('inds_labels_all:',inds_labels_all)

# 1. Nans
# mask of valid label indices
# mask_labels_valid = np.in1d(inds_labels_all,self.labels_valid)
mask_labels_valid = np.in1d(inds_labels_all,ex_labels_valid)
print('mask_labels_valid:',mask_labels_valid)

# replace invalid with nans
inds_labels_all[~mask_labels_valid] = np.nan
print('inds_labels_all:',inds_labels_all)
# inds_labels_valid = inds_labels_all[~np.isnan(inds_labels_all)]
# print('inds_labels_valid:',inds_labels_valid)

# 2. Indices
# mask of valid toocan labels
# mask_toocan_labels_valid = np.in1d(self.labels_toocan,self.labels_valid)
mask_toocan_labels_valid = np.in1d(ex_labels_toocan,ex_labels_valid)
print('mask_toocan_labels_valid:',mask_toocan_labels_valid)
toocan_labels_valid_unordered = ex_labels_toocan[mask_toocan_labels_valid]
print('toocan_labels_valid_unordered:',toocan_labels_valid_unordered)
order_toocan_valid = sorted(range(len(toocan_labels_valid_unordered)), key=lambda k: toocan_labels_valid_unordered[k])
print('order_toocan_valid:',order_toocan_valid)

# find indices of valid toocan labels
ind_toocan_valid = np.where(mask_toocan_labels_valid)[0]
print('ind_toocan_valid:',ind_toocan_valid)
# assign 
inds_labels_all[mask_labels_valid] = np.take(np.array(ind_toocan_valid,dtype=int),order_toocan_valid)
print('inds_labels_all:',inds_labels_all)
inds_labels_all = np.asarray(inds_labels_all,dtype=int)
# inds_labels_all[inds_labels_all < 0] = np.nan
print('inds_labels_all:',inds_labels_all)

print('lab,ind')
for lab in ex_labels_valid:
    
    print(lab, inds_labels_all[lab])